<a href="https://colab.research.google.com/github/oviyajay/ML-Final-Project/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import np from numpy
import tensorflow as tf
import os 
import cv2
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/
#image preprocessing

TEST_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/with_mask"
TEST_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/without_mask"
TRAIN_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/with_mask"
TRAIN_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/without_mask"


In [ ]:
#Eric 

In [ ]:
#Iant

In [ ]:
#Anindita

In [ ]:
#Oviya

